In [ ]:
import json

PATH_PREGUNTAS = "preguntas/preguntas.json"   ##CAMBIAR
PATH_PROMPTS = "prompts/prompts.json"
API_KEY = "YOUR API KEY"
ASSISTANT_ID = "YOUR ASSISTANT ID KEY"

# Abre el archivo JSON y carga los datos
with open(PATH_PREGUNTAS, 'r') as file:
    data = json.load(file)

preguntas = []
respuesta_simulada = []

# Itera sobre cada pregunta en los datos
for pregunta in data['preguntas']:
    # Extrae la información relevante de la pregunta
    id_pregunta = pregunta['id']
    categoria = pregunta['categoria']
    texto_pregunta = pregunta['pregunta']
    opciones = pregunta['opciones']
    opcion_a = pregunta['opciones'][0]["texto"]
    opcion_b= pregunta['opciones'][1]["texto"]
    opcion_c = pregunta['opciones'][2]["texto"]
    opcion_d = pregunta['opciones'][3]["texto"]
    respuesta_correcta = pregunta['respuesta_correcta']
    respuesta_simulada.append(respuesta_correcta)
    
    # Agrega la pregunta a la lista de preguntas
    preguntas.append(

        "Elije la respuesta a la pregunta formulada de entre las tres opciones posibles. Devuelve dos elementos separados por un acento circunflejo, cuyo primer elemento sea el id de la respuesta y el segundo una justificación.\n" + texto_pregunta +"\na)"+ opcion_a +"\nb)"+ opcion_b +"\nc)"+ opcion_c+"\nd)" + opcion_d

    )

# Imprime la lista de preguntas para verificar
#print(preguntas)

In [2]:
from openai import OpenAI

#Creo el cliente
client = OpenAI(api_key=API_KEY)


#Definimos funciones para crear el hilo, lanzarlo, esperar y recibir el mensaje
def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )

def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(ASSISTANT_ID, thread, user_input)
    return thread, run

import time

# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


# Waiting in a loop
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [3]:
#ESPERAR HASTA QUE TERMINE DE EJECUTARSE LA CELDA POR COMPLETO

#Prompt que usamos
prompt = "Elije la respuesta a la pregunta formulada de entre las tres opciones posibles. Devuelve dos elementos separados por un acento circunflejo, cuyo primer elemento sea el id de la respuesta y el segundo una justificación."

resultados = []
a=0

#Vamos lanzando los hilos y almacenando las respuestas
for i in preguntas:
    thread, run = create_thread_and_run(i)
    run = wait_on_run(run, thread)
    pretty_print(get_response(thread))
    response = get_response(thread)

    response = get_response(thread)

    # Imprimir mensajes
    print("# Messages")
    for message in response:
        role = message.role
        content = message.content[0].text.value
        print(f"{role}: {content}")

        if role == "user":
            mensaje_user = content

        if role == "assistant":
            mensaje_asistente = content

    resultados.append({
        "pregunta": mensaje_user,
        "prompt":prompt,
        "respuesta_simulada": respuesta_simulada[a],
        "respuesta_correcta": mensaje_asistente.split("^")[0].strip(),
        "justificacion_simulada": mensaje_asistente.split("^")[1].strip()
    })
    a=a+1

print(resultados)

#ESPERAR HASTA QUE TERMINE DE EJECUTARSE LA CELDA POR COMPLETO

# Messages
user: Elije la respuesta a la pregunta formulada de entre las tres opciones posibles. Devuelve dos elementos separados por un acento circunflejo, cuyo primer elemento sea el id de la respuesta y el segundo una justificación.
¿Cuánto es 2 + 2?
a)3
b)4
c)5
assistant: b^En matemáticas, la suma de 2 + 2 resulta en 4.

# Messages
user: Elije la respuesta a la pregunta formulada de entre las tres opciones posibles. Devuelve dos elementos separados por un acento circunflejo, cuyo primer elemento sea el id de la respuesta y el segundo una justificación.
¿Cuánto es 2 + 2?
a)3
b)4
c)5
assistant: b^En matemáticas, la suma de 2 + 2 resulta en 4.
# Messages
user: Elije la respuesta a la pregunta formulada de entre las tres opciones posibles. Devuelve dos elementos separados por un acento circunflejo, cuyo primer elemento sea el id de la respuesta y el segundo una justificación.
¿Cuánto es 5 - 3?
a)2
b)3
c)4
assistant: a^La operación matemática 5 - 3 resulta en 2.

# Messages
user: Elije 

In [4]:
# Guardar los resultados en un archivo de texto
with open("resultados.txt", "w") as file:
    for resultado in resultados:
        file.write(f"Pregunta: {resultado['pregunta']}\n")
        file.write(f"Respuesta correcta: {resultado['respuesta_correcta']}\n")
        file.write(f"Respuesta simulada: {resultado['respuesta_simulada']}\n")
        file.write(f"Justificación simulada: {resultado['justificacion_simulada']}\n\n")